# Recipe: Reusing a Chemical Dataset. Part 1- Finding Datasets
## Scenario
Our group has a set of thermophysical data on over 8000 chemical substances.  We want to integrate into this dataset another physical property dataset so that we can do an analysis of the correlations of the thermophysical data with the chosen physical property of the substances (that are common to both sets).

Criteria for picking the physical property dataset: high quality, trusted, large, available with an open license, so I can publish the results and make the derived dataset open.
- **High quality means**: unambiguous identification of each chemical substance, enough contextual information (metadata) to make the values scientifically useful, i.e., at least the composition of the solvent, the temperature and for volatile substances the pressure.
- **Trusted means**: the provenance chain is reported with the data, and it shows that the data comes from a reputable source(s) and any aggregation and/or processing is documented in enough detail that the community can understand how the dataset has been created/provided.

## Searching PubChem for datasets
Pubchem houses a lot of data about chemical substances, compounds and bioassays.  Over time external organizations have worked with
PubChem to include data, in one of a couple of ways:
- data that has been integrated into PubChem pages (e.g., [CCDC](https://pubchem.ncbi.nlm.nih.gov/source/941) -> [example](https://pubchem.ncbi.nlm.nih.gov/compound/241))
- data that is not available in a PubChem page but is available via the data sources section of the site as 'annotations' (e.g. [RCSB PDB](https://pubchem.ncbi.nlm.nih.gov/source/15751) -> [Example](https://pubchem.ncbi.nlm.nih.gov/source/15751#data=Annotations))

The data available is may not be structured and or clearly described, however if the source has a website with an API then you are likely to get better quality metadata from the linked site.

In [ ]:
import requests
import json
# This URL is the metadata about the data sources in PubChem
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/sourcetable/all/JSON/?response_type=display'
response = requests.get(url)
srcs = response.json()
results = []
search = 'Curation Efforts'  # i.e., a repository, or other type of data source (this is in index 8 of the data list for each source)
rows = srcs['Table']['Row']
for row in rows:
    if row['Cell'][8].find(search) != -1:
        hit = {}
        hit.update({'name': row['Cell'][0]})
        hit.update({'url': row['Cell'][9]})
        results.append(hit)
print(json.dumps(results, indent=4))

## Searching FAIR Sharing for datasets
FAIRSharing is a database of FAIR resources and per se a database of datasets, however you might find a repository here that
has the kind of data you are looking for. The code below a

In [ ]:
from hiddensettings import *
import requests
import json
# see https://fairsharing.org/API_doc for instructions on how to search the API
# user login
url = 'https://api.fairsharing.org/users/sign_in'
loghdrs = {'Accept': 'application/json','Content-Type': 'application/json'}
login = {'user': {'login': fs_user, 'password': fs_pass}}
response = requests.request("POST", url, headers=loghdrs, data=json.dumps(login))
data = response.json()
jwt = data['jwt']
srchdrs = {'Accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': "Bearer {0}".format(jwt)}
searchterm = 'chemistry'
searchurl ='https://api.fairsharing.org//search/fairsharing_records?q=' + searchterm
search = requests.request("POST", searchurl, headers=srchdrs)
hits = json.loads(search.content)
for hit in hits['data']:
    print(hit['attributes']['name'] + ": " + hit['attributes']['url'])

Author: Stuart Chalk
Date: Dec. 12, 2022